# PDAML Project 1: Using Machine Learning to Identify Trends in Scotland's 2011 Census

In this project machine learning was used to try to identify trends in scotlands 2011 Census. The Census divides Scotland into different sectors. There are a variety of ways of dividing up the sectors from the biggest (Scottish parliament regions) which divide scotland into only 8 regions to the smallest (Output area) which are the building blocks of the census study and divide scotland into 46351 areas.In this study I will divide scotland into the local charactersitic (LC) postcode sectors. These combine the postcodes with the same identifier except the last two characters. For example EH1 1(XX). There are 1012 of such sectors around Scotland, which means there are enough of them to train and test a machine learning algorithm on them.

In [ ]:
import census_utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, tree, metrics, model_selection, ensemble, neural_network
from IPython.display import Image

In [ ]:
pd.set_option('display.max_colwidth', -1)
lookup = pd.read_pickle('../lookup/lookuptable.pkl')
lookup.head()

In [ ]:
lookup.shape

# Target Feature:

The target feature I will try to predict in this study is the National Statistics Socio-economic Classification(NS-SeC). This feature divides the population into different classifications. Out of the different classifications I have selected five:

0002: Higher managerial, administrative and professional occupations     
0003: Lower managerial, administrative and professional occupations    
0004: Intermediate occupations                  
0006: Lower supervisory and technical occupations      
0008: Routine occupations

I have chosen these five classifications because they follow a hierarchical pattern, and I assume there is a large correlation between the classifications and their salary. I have decided to leave out Small employers and own account workers because it does not fit into the hierarchical pattern. I also did not include full time students and long term unemplyed because in my study I am interested in the working population.

The individual LC postcode sectors were then split up into 5 different classes based on the weighted average of their target feature. Below are displayed the histograms and the choropleth maps of the target feature just based on the score and when split into the five classes.

In [ ]:
requested = { 'LC6114SC': ['0002', '0003', '0004', '0006', '0008'] }
weights = { 'LC6114SC0002' : 5, 
           'LC6114SC0003' : 4,
           'LC6114SC0004' : 3,
           'LC6114SC0006' : 2,
           'LC6114SC0008' : 1 }
tiers = [0.1, 0.2, 0.5, 0.8]
outcome = census_utils.genclfscore(requested, weights, tiers)

In [ ]:
outcome.head()

In [ ]:
outcome.Score.hist()

In [ ]:
copts = { "fill_color" : "YlGn", "legend_name" : "NS-SeC Score"}
mopts = { "location" : [55.95, -3.19], "zoom_start" : 8}
census_utils.genchoropleth(outcome, 'Score', copts_custom=copts, mopts_custom=mopts)

In [ ]:
outcome.Class.hist()

In [ ]:
copts = { "fill_color" : "YlGn", "legend_name" : "NS-SeC Class"}
mopts = { "location" : [55.95, -3.19], "zoom_start" : 8}
census_utils.genchoropleth(outcome, 'Class', copts_custom=copts, mopts_custom=mopts)

# Prediction Features:

In order to predict the target feature (NS-SeC) 5 features were chosen. These were:             

Highest qualification: level 4 or above                       
General health: Very good health                      
Car or van availability: two or more cars/vans per household                       
Accomodation type: Owned outright                                
Distance travelled to work: 30 km or more                     

These were chosen because I assumed the correlation between these features and the tier of the NS-SeC.

High qualification for higher tier jobs, higher tier jobs encouraging better health, higher tier job meaning higher salary therefore more money availible to buy cars and own houses. And lastly higher tier job being worth commuting to over long distances.

These all had to be normalised with either total population in that specific group, for example to obtain the proportion of people with level 4 qualification or above. For example he number of people with level 4 qualification or above was divided with the total population aged 16 or over.

In [ ]:
#LC5102SC Highest qualification by age 006: level 4 and above
#LC4109SC Car or van availability 004: two or more cars per household
#LC3102SC General health 002: very good health
#LC4427SC accomodation type 003: owned outright
#LC7102SC distance travelled to work 008: 30 km or more

requested = { 'LC5102SC': ['0006'],
              'LC4109SC': ['0004'],
              'LC3102SC': ['0002'],
              'LC4427SC': ['0003'],
              'LC7102SC': ['0008'] }
norm = { 'LC5102SC' : { '0006':'0001' },
         'LC4109SC' : { '0004':'0001' },
         'LC3102SC' : { '0002':'0001' },
         'LC4427SC' : { '0003':'0002' },
         'LC7102SC' : { '0008':'0001' }}

features = census_utils.extractdata(requested, norm)

In [ ]:
features.head()

In [ ]:
features.shape

In [ ]:
# extract data and rarget 
data = features.values
target = outcome['Class'].values 
data.shape, target.shape

In [ ]:
target[0:5], features.columns.values

In [ ]:
#2d plots showing the feature distribution
classes = [1, 2, 3, 4, 5]
flabels = ['Level 4 qualification or above', 'Owns two or more cars', 'Very good health', 'Accomodation owned outright','Travels 30 or more km to work']
clabels = ['Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
census_utils.featureplot(data, target, classes, flabels, clabels)

A choropleth map of Proportion of Level 4 qualification or above can be compared to that of NS-SeC class. A vague correlation is seen between the two maps. With high proportion of level 4 qualification or above in central Edinburgh, Glasgow and Aberdeen. These regions also have high NS-SeC Class. While the regions to the South of Glasgow and between Edinburgh and Glasgow both have a low Proportion of Level 4 qualification or above and also have low NS-SeC Class.

In [ ]:
copts = { "fill_color" : "YlGn", "legend_name" : "Proportion of Level 4 qualification or above"}
mopts = { "location" : [55.95, -3.19], "zoom_start" : 8}
census_utils.genchoropleth(features, 'LC5102SC0006', mopts_custom=mopts, copts_custom=copts)

Before even applying machine learnin to the data several interesting facts can be observed from looking at the 2d plots.

1) Proportion of population with level 4 qualification or above, or very good health are very good at seperating the NS-SeC target feature into different tiers. In both cases the individual data points are well seperated based on the NS-SeC tier.

2) The other features are pretty poor at seperating the NS-SeC target feature into different tiers.

3) There is not a lot of data for NS-SeC tier 2.

4) There appears to be a positive correlation between owning 2 or more cars and very good health, as well as owning 2 or more cars and having a level 4 qualification or above. Both of these are interesting because I would not expect there to be much correlation between them.

5) Lastly an interesting plot is that between owning 2 or more cars and traveling to work 30 km or more. There seems to be a very slight correlation but not to the point I would expect. I assume this is because owning 2 or more cars is more of a commodity and not a necessary thing to travel to work. This graph however does not seperate the target data into its individual tiers. therefore it is not very useful for machine learning.

# Machine learning

In order to predict the target feature which in this case is NS-SeC Class from the input feature a decision tree is used. First of all the data is split up into train and test data with 70% being used to train and the remaining 30% being used for test. First of all a decision tree is trained using the data reserved for training. No restrictions were applied to the decision tree which is the reason for the large number or leaf nodes and deapth.

In [ ]:
train_data, test_data, train_target, test_target = model_selection.train_test_split(
    data, target, test_size=0.3, random_state=0)

print(train_data.shape, train_target.shape, test_data.shape, test_target.shape)

In [ ]:
clf = tree.DecisionTreeClassifier()
fit = clf.fit(train_data, train_target)

In [ ]:
# plot decision tree 
print(flabels, classes)
graph = census_utils.plotDT(fit, flabels, clabels)
Image(graph.create_png())

In [ ]:
# display classification report 
expected = test_target
predicted = clf.predict(test_data)
report = metrics.classification_report(expected, predicted)
print(report)

# define and plot confusion matrix
cm = metrics.confusion_matrix(expected, predicted)
census_utils.plot_cm(cm, clabels)
plt.show()

In [ ]:
# Cross validation 
scores = model_selection.cross_val_score(clf, data, target, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

The classification report, confusion matrix and cross validation use the 30 % of the data to test the decision tree. Precision of 85% is observed for tier 1, 71% for tier 3, 75% for tier 4 and 79% for tier 5. However a precision of only 50% for tier 2. The reason for this could be because of two things. First that tier 2 in not a well defined NS-SeC therefore it is difficult to distinguis it from the other tiers. And the second that there is not a large amount of datapoints for it since only a total of 92 data points were in the tier 2 NS-SeC and that is before splitting it into train and test data.

However the decision tree preforms well with an overall acuracy of 66% +/- 8%.

In [ ]:
#redefine the train test data split
train_data, test_data, train_target, test_target = model_selection.train_test_split(
    data, target, test_size=0.3, random_state=0)

# chose the hyper parameters over which to grid-search
params = {'max_leaf_nodes' : [2, 3, 4, 5, 10, 20, 50], 
          'max_depth' : [2, 3, 4, 5, 6, 7, 8]
         }
score = 'accuracy'
clf = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), params, cv=5, scoring='%s' % score)
clf.fit(train_data, train_target)


print(clf.best_params_)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

In [ ]:
# heatmap used to display the best number of max leaf nodes and max depth
labels = [k for k,v in params.items()]
classes = [v for k,v in params.items()]
census_utils.heatmap(means.reshape(7,7), classes, labels, palette="Red")
plt.show()


I then use a grid search to find the best fit parameters for max leaf nodes and max depth. The output values for the best fit parameters are 20 for max leaf nodes and 4 for max depth. Note by looking at the heatmap it can be seen that anything with more than 10 max leaf nodes will preform well. However having max leaf nodes over 20 does not improve the performance of the decision tree. This is because the decision tree does not have to use the maximum number, but just the number that preforms well enough.

The decision tree with the new best fit parameters is displayed below and its accuracy comes out to 67% +/- 10% which is barley better than the initial decision tree. However this decision tree is much smaller and is therefore much easyer to interpret, making it more useful.

In [ ]:
#definig a classification decision tree with new best fit parameters
clf = tree.DecisionTreeClassifier(max_depth = 4,max_leaf_nodes = 20)

# run the classification using the training data
fit = clf.fit(train_data, train_target)


graph = census_utils.plotDT(fit, flabels, clabels)
Image(graph.create_png())

In [ ]:
# cross validation for new decision tree
clf = tree.DecisionTreeClassifier()
scores = model_selection.cross_val_score(clf, data, target, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# Conclusion

The aim of this project was to predict a target feature given 5 input features using machine learning. The Scotland's 2011 Census data was used to try to predict the NS-SeC which was split into 5 tiers based on the jobs hierarchical position and salary. Five input features that appeared to have a slight correlation to the NS-SeC tier were chosen such that they would be able to predict the tier of the NS-SeC based on their values.

My initial beliefs on features such as Very good health and level 4 qualification or above being good predictors of the NS-SeC tier were confirmed. However other features that I believed would be good at predicting the NS-SeC tier such as owning two or more cars, owning property and commuting 30 km or more were poor at predicting the tier.

Interesting correlations were observed between very good health and owning two or more cars. And an even more interesting lack of correlation between owning two or more cars and commuting 30 km or more to work.

The dataset could have been improoved by having more datapoints for tier 2 of NS-Sec which is that of Lower supervisory and technical occupations. Having more samples for that datapoint would have make the classifier better trained. Improvements could have also been make when looking at normalising input features. A more precise part of the population could have been used to normalise these input features. For example when looking at the proportion of population with level 4 qualification or above, it was normalised by dividing by the total population above 16 years old. However level 4 qualification can be acheved much later in the mid 20s. Therefore a more accurate normalisation would be by dividing by total population above say 24 years old.

Another shortcoming of the data used is that there is a mix of data from cities and rural data. The differences in jobs availibe between the two does have an affect on the data. Therefore it would have been better to use seperate data and construct seperate decision trees for urban and rural areas. Also here we must remember that we are looking at regions (LC postcode sectors). Which means that the data is generalised over that specific region. The bigger the region the greater the generalisation.

The study preformed here could be used in several policies. The first being that it could be used to increase government spending on education. Even though Scotland does have great education with local students receiving university education for free, improvements could be made. For example promoting continuing studies to masters and doctorate studies, as those result in higher paying jobs. This could be extended to health, saying that higher level of qualification also results in better health. Secondly this study could be used to promote not buying cars and using more enviromentally friendly methods of transportation. As the study showed car ownsership does not correlate well with commute distance, which suggests that cars are commodities and are not a neccesary possesion.